# Tutorial Searching for Ground States.
In this tutorial, we learn how to use the FTPS solver to check if the ground state we found is correct. The Density Matrix Renormalization Group (DMRG) is a variational method and hence can get stuck in local minima. We use DMRG at two points in the solver. Once to find the sector of particle number in which the ground state is and in a second run to calculate the true ground state in the sector found.

Experience with the solver tells us that most problems occur in insulating systems, so let us set up a simple one-orbital insulating AIM.

In [ ]:
#%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np

# ForkTPS
from forktps.DiscreteBath import *
from forktps.Helpers import *

from triqs.gf import *
from triqs.operators import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

def EmptySolver(bath, e0):
    S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-10., wmax=10.)
    S.b, S.e0 = bath, e0
    
    return S

In [ ]:
Norbs = 1
gfstruct = [['up',[0]], ['dn',[0]]]

# Interaction strength
U = 6
# Bath size
Nb = 10


# Bath hybridization consists of two semi-circles at -3 and 3 with a gap between [-2, 2]
delta = GfReFreq(indices = [0], window = (-10., 10.), n_points =  10001, name = "up")
delta['0','0'] << SemiCircular(half_bandwidth = 1.0, chem_potential= -3) + SemiCircular(half_bandwidth = 1.0, chem_potential=3)
Delta = BlockGf(name_list = ('up','dn'), block_list = (delta,delta), make_copies = True)


# Discretize hybridization, gap tells the function to place no bath sites between -2 and 2 otherwise DMRG would not work.
bath = DiscretizeBath(Delta, Nb, gap=[-2.,2.] )

# Calculate the discrete hybridization and compare it.
DeltaRe = bath.reconstructDelta( w = delta.mesh, eta = 0.1 )
plt.figure()
for j in range(1):
    plt.plot(getX(Delta.mesh), -Delta  ['up'][j,j].imag.data, label = 'Original indx ' + str(j))
    plt.plot(getX(Delta.mesh), -DeltaRe['up'][j,j].imag.data, label = 'Reconstruction indx ' + str(j))
    
    
e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill([[-U/2]]) # put the same 1x1 matrix on both blocks; set to half filling

# Interaction
Hint = ftps.solver_core.HInt(u=U)

Now let us start the solver. To check the ground state, let us use the measurements parameter for the solve call. It is a list of measurements where each measurement is a TRIQS many-body operator acting on the impurity sites in the FTPS tensor network.

In [ ]:
# Initialize the Solver
S = EmptySolver(bath, e0)

# we want to look at the ground state only so set time_steps to 0
tevo = ftps.solver.TevoParams(time_steps = 0)
dmrg = ftps.solver.DMRGParams(sweeps = 10)

# measure: occupation of first orbital spin up, occupation of first orbital spin down and double occupancy.
measurements = [ n("up",0), n("dn",0), n("up",0)*n("dn",0) ]

S.solve( h_int = Hint, 
         tevo = tevo, 
         params_partSector = dmrg,
         params_GS = dmrg,
         measurements = measurements
         )

# The result of the measurement can be found in S.measurment_results
for indx, meas in enumerate(measurements):
    print(f'Measurement of Operator: {meas} resulted in {S.measurement_results[indx]:.3}' )
    
print(f'\nGround state is in the sector{S.NPartGS}')

Obviously something is suspicious here. We have a spin-symmetric model but one of the spins is nearly totally occupied while the other is nearly empty. So the ground state the solver found is highly spin-polarized and breaks the spin-symmetry of the model.
By using the NPartGS member of the Solver, we also see that the total number of particles (impurity and bath combined) in the spin-up and spin-down channel are different, explaining at least why we observe different occupations for the two spin-channels. This leaves us with three possibilities:

A.) There really is some form of symmetry breaking in this model and the solver found the correct ground state. <br>
B.) The search for the ground state sector failed and we were looking at the wrong one. <br>
C.) It could be the correct sector, but DMRG in this sector did not work. <br>

Ultimately, since DMRG is a variational method, the only way to be 100% certain about the correct sector is to solve the problem with another, non-variational solver. That said, DMRG works very well in most cases and as long as certain quantities do not look completely off, the result can be trusted in general. In the following we would like to discuss a couple of approaches how to increase the confidence in the ground state.

# Search in surrounding Sectors
The FTPS solver has two modes how to find the ground state sector. The first one (used by default) is to perform a DMRG calculation without particle number conservation and hope it will turn out in the correct sector. The second possibility is for the user to provide a list of sectors in which the solver should search and it then takes the one with the lowest energy.

Since we know the sector the solver found, let us search in a couple of surrounding sectors (experience and physical intuition tells us that even if the solver finds the wrong sector, it is not far from the true ground-state sector). This can be done with the _NPart_ argument in the solve call:

In [ ]:
# Initialize the Solver
S = EmptySolver(bath, e0)

# Look through sectors 5,5 ... 5,6 ... 6,5 ... 6,6 .......
# NPart needs to be a list of lists with 2*Norbs entries
NPart = [[5,5], [5,6], [6,5], [6,6], [6,7], [7,7]]

measurements = [ n("up",0), n("dn",0), n("up",0)*n("dn",0) ]
tevo = ftps.solver.TevoParams(dt = 0.1, time_steps = 0)
dmrg = ftps.solver.DMRGParams(sweeps = 10)

S.solve( h_int = Hint, 
         tevo = tevo, 
         params_partSector = dmrg,
         params_GS = dmrg,
         measurements = measurements,
         NPart = NPart, # here we define in which sectors to look for the ground state
         )

# The result of the measurement can be found in S.measurment_results
for indx, meas in enumerate(measurements):
    print( 'Measurement of Operator: ',meas,' resulted in %.3f'%S.measurement_results[indx] )
    
print('Ground state is in the sector' ,*S.NPartGS)

# We can also view the energies and Impurity Occupations of the various sectors provided
print('\nChecking All Sectors:')
Es = S.SectorEnergies
Occs = S.SectorImpOccs
for indx, [e, occ] in enumerate( zip( Es, Occs ) ):
    print(f"In sector {NPart[indx]}, energy: {e:.4},  ImpOcc: {occ}" )

We notice:

1. Again we find the same ground state sector (maybe the spin reversed one).
2. We see that there exist two degenerate ground states in sectors [5,6] and [6,5]. Would we add them together, we would find the expected spin-symmetry and a half filled impurity.
3. The energy of the other sectors is not close to the ground-state but very different (gap > 1.5).
4. The behavior of every sectors make sense. Increasing the total number of particles also increases the impurity occupation.
5. Sectors [5,5] and [6,6] have the same energy, hence there is some symmetry and averaging over them would yield a half filled, symmetric impurity. Again a reflection of particle-hole symmetry

This is already a strong indication that we found the true ground state. But we can do more:

# Preparation before DMRG

We can improve the convergence of DMRG by providing it with a state to start with that is closer to the true ground state. For this the solver has implemented two different methods:

1. __Apply the Hamiltonian:__ <br>According to the Power method $\lim_{n \to \infty} H^n | x\rangle$ is proportional to the state with extremal energy, provided $| x \rangle$ is not orthogonal to it. This helps with DMRG convergence also in cases where this state is not the ground state because the starting point is mixed up by the Hamiltonian. <br> <br>
2. __Imaginary Time evolution:__ <br> Similarly, $e^{-\tau H } | x\rangle$ becomes proportional to the ground state in the limit $\tau \to \infty$. While DMRG is by far superior for finding the exact ground state, imaginary time evolution can be used to create a better starting point.

By default, the Hamiltonian is applied a couple of times, but no imaginary time evolution is performed. We can change this using the preparation parameters in DMRGParams:

In [ ]:
# Initialize the Solver
S = EmptySolver(bath, e0)

NPart = [[5,5], [5,6], [6,5], [6,6], [6,7], [7,7]]
measurements = [ n("up",0), n("dn",0), n("up",0)*n("dn",0) ]
tevo = ftps.solver.TevoParams(dt = 0.1, time_steps = 0)

# Set parameter for preparation before DMRG. This can be done separately for DMRG during sector search 
# as well as for DMRG after the sector is found. Here we use the same parameters in both.
dmrg = ftps.solver.DMRGParams( sweeps = 10,   
                               prep_napph = 5,           # Number of times H is applied.
                               prep_imagTevo = True,     # Turns on imaginary time evolution
                               prep_dtau = 0.5,          # Time step size (choose a rather large number)
                               prep_time_steps = 10,     # Number of time steps
                               prep_method = 'TDVP' )    # Time evolution method. Note TDVP has problems starting
                                                         # with product states, but H is applied before imaginary
                                                         # time evolution, so this is not a big problem. 

S.solve( h_int = Hint, 
         tevo = tevo, 
         params_partSector = dmrg,
         params_GS = dmrg,
         measurements = measurements,
         NPart = NPart, # here we define in which sectors to look for the ground state
         )

# The result of the measurement can be found in S.measurment_results
for indx, meas in enumerate(measurements):
    print('Measurement of Operator: ', meas,' resulted in %.3f'%S.measurement_results[indx] )
    
print('Ground state is in the sector' ,*S.NPartGS)

# We can also view the energies and Impurity Occupations of the various sectors provided
print('\nChecking All Sectors:')
Es = S.SectorEnergies
Occs = S.SectorImpOccs
for indx, [e, occ] in enumerate( zip( Es, Occs ) ):
    print(f"In sector {NPart[indx]}, energy: {e:.4},  ImpOcc: {occ}" )

Again we find the exact same ground state and also the same energies and occupations in all the sectors. Another indication that everything went as expected.

# Decrease bath size
The last thing we can do to check everything is fine is to simply decrease the bath size and check if the ground state is similar by looking for example at the impurity occupation (note that the ground state energy scales $\sim N$, hence it cant be used to compare different bath sizes). Generally smaller bath sizes are easier convergence in DMRG. Note however that this can only increase our confidence in the ground state of the larger sector. In case these states differ by a lot it does not necessarily mean that something went wrong. 

In [ ]:
# Various bath sizes
Nbs = [4, 6 , 8, 10]

measurements = [ n("up",0), n("dn",0), n("up",0)*n("dn",0) ]
tevo = ftps.solver.TevoParams(dt = 0.1, time_steps = 0)
dmrg = ftps.solver.DMRGParams( sweeps = 10 ) # use default preparation (no imaginary time evolution)

for Nb in Nbs:
    bath = DiscretizeBath(Delta, Nb, gap=[-2.,2.])
    
    # Initialize the Solver
    S = EmptySolver(bath, e0)
    
    S.solve( h_int = Hint, 
             tevo = tevo, 
             params_partSector = dmrg,
             params_GS = dmrg,
             measurements = measurements
             )

    print(f'\nResult for Nb = {Nb}')
    print('-------------------------')
    print(f'Ground state energy: {S.E0:.4}')
    # The result of the measurement can be found in S.measurment_results
    for indx, meas in enumerate(measurements):
        print(f'Measurement of Operator: {meas} resulted in {S.measurement_results[indx]:.3}' )

    print(f'Ground state is in the sector {S.NPartGS}')

All bath sizes show the exact same behavior: the ground state we find is spin-polarized in a sector of particle number where one spin is larger than the other. Depending on details of the solver, DMRG might find any of the two degenerate ground states (on the computer where this tutorial was written, executing the above code about 3 times resulted in one bath size with the other ground state).

It is important to note that any one of the two is a valid ground state. Adding them up in the tensor network does not make any sense because it would unnecessarily increase the bond dimension and it wouldn't even be possible in the first place because the resulting state is no eigenstate of $S^z$ with defined magnetization.

Further prove that these two states really are equivalent and how to restore the symmetry when one would like to perform a paramagnetic calculation can be found in the tutorial:<br> *RestoringSymmetry.ipynb*